In [ ]:
!pip install openpyxl

In [ ]:
## Chapter04から使用するテキストデータをコピーする（事前にChapter04/Sentiment.ipynbを実行してください）
!cp -r ../Chapter04/text ./
!cp -r ../Chapter04/sisyou_db ./

In [ ]:
import spacy
from spacy.matcher import Matcher
import pandas as pd

nlp = spacy.load("ja_ginza")
matcher = Matcher(nlp.vocab)
df = pd.read_excel("sisyou_db/sisyou_db_h29_01.xlsx", skiprows=[1])
df.head()

In [ ]:
patterns = [[{"POS": "NOUN"}] * n for n in [2,3,4]]

## GiNZA 5.0.X
#for pattern in patterns:
#    name = f'noun_phrase_{len(pattern)}'
#    matcher.add(name, [pattern])

## GiNZA 4.0.X
 for pattern in patterns:
     name = f'noun_phrase_{len(pattern)}'
     matcher.add(name, None, pattern)

In [ ]:
texts = [
    "工場内で、鉄骨階段（仮組）高さ2m50cmに手摺を仮組立作業中、足を滑らせて転倒し、顔面を強打して骨折し、右脇腹肋骨も骨折した。",
    "倉庫の出入口の階段を荷物（冷凍商品15kgぐらい）を持って下りる際に、階段が凍っていて滑って転倒し、階段を転げ落ち（4段位）、持っていた荷物を足に落としてしまい、右足の腓骨を骨折した。",
    "道路の3車線の真ん中を走行中、左車線に侵入してしまい、走行中の大型ワンボックスカーと衝突し、首と左肩を痛め、回転性のめまいで入院し、痺れもある。"
]

for doc in nlp.pipe(texts):
    print(doc.text)
    for match_id, begin, end in matcher(doc):
        print(nlp.vocab.strings[match_id], doc[begin:end])
    print('='*80)

In [ ]:
from collections import Counter

counter = Counter()
for doc in nlp.pipe(df["災害状況"]):
    nps = [doc[begin:end].text for _, begin, end in matcher(doc)]
    counter.update(nps)

In [ ]:
print('count word')
for word, count in counter.most_common(200):
     print('{}\t{}'.format(word,count))